### Analyse des données initiales

In [1]:
import json
import os
import re
from typing import Dict, List, Tuple
import pandas as pd

In [2]:
SOURCE_FOLDER = "../data/INSALyon/insalyon-2023-12-13-page/"
source_files = os.listdir(SOURCE_FOLDER)

In [3]:
def read_json_file(filename):
    with open(os.path.join(SOURCE_FOLDER, filename), 'r') as f:
        read_file = json.load(f)
    return read_file


def get_title_and_text(file_content):
    return pd.Series([file_content["hasPart"][0]["title"], file_content["hasPart"][0]["text"]])


def get_chunk_length(text):
    return pd.Series([len(text), len(text.split())])

In [4]:
source_data = pd.DataFrame(source_files, columns=["filename"])

In [5]:
source_data["file_content"] = source_data["filename"].apply(read_json_file)

In [6]:
source_data[["title", "text"]] = source_data["file_content"].apply(get_title_and_text)

In [7]:
source_data[["chunk_len_chars", "chunk_len_words"]] = source_data["text"].apply(get_chunk_length)
source_data["chunk_len_words"].describe()

count      47.000000
mean      340.553191
std       530.366027
min         4.000000
25%        15.500000
50%       287.000000
75%       437.500000
max      3497.000000
Name: chunk_len_words, dtype: float64

Problems :
- Some chunks ave low amount of word
- variation in chunk size is high
- some chunks are too big
- some chunks need cleaning (symbols and links everywhere)
- some chunks are just links toward other pages (ladrome-page-558941.json)

Ok stuff :
- The scrapping and parsing of the webpages is actually good (no messy stuff)

In [8]:
sorted_chunks = source_data.sort_values(by="chunk_len_words")
sorted_chunks.head(5)

,filename,file_content,title,text,chunk_len_chars,chunk_len_words
19,rejoindreinsalyon-page-2893.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Visite du département informatique,Visite du département informatique\n\n,36,4
12,rejoindreinsalyon-page-2871.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Visite du département matériaux,Visite du département matériaux\n\n,33,4
10,rejoindreinsalyon-page-2864.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Visite du département génie électrique,Visite du département génie électrique\n\n,40,5
18,rejoindreinsalyon-page-2890.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Visite du département génie industriel,Visite du département génie industriel\n\n,40,5
14,rejoindreinsalyon-page-2877.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Visite du département génie mécanique,Visite du département génie mécanique\n\n,39,5


In [9]:
sorted_chunks.tail(5)

,filename,file_content,title,text,chunk_len_chars,chunk_len_words
37,rejoindreinsalyon-page-3908.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Roxane,Roxane\n\n> Le tout est de trouver **son équil...,3796,647
2,rejoindreinsalyon-page-160.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Vie de campus,Vie de campus\n\nLa vie sur le campus\n-------...,7164,734
5,rejoindreinsalyon-page-2483.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",International students,"International students\n\nOur institution, you...",7784,759
8,rejoindreinsalyon-page-282.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Mentions légales,Mentions légales\n\nSelon les termes de la loi...,6234,900
25,rejoindreinsalyon-page-3523.json,"{'@context': 'https://wikit.ai', '@type': 'Doc...",Quiz INSA,Quiz INSA\n\n0%\n\n \n\n![Campus INSA Lyon](ht...,50092,3497


In [13]:
HTML_FOLDER = "../data\D26\ladrome-2023-12-22-page/"
#FILE = "ladrome-page-480456.json"
#FILE = "ladrome-page-10516.json"
#FILE = "ladrome-page-567239.json"
#FILE = "ladrome-page-573222.json"
#FILE = "ladrome-page-555850.json"
FILE = "ladrome-page-581681.json"
with open(os.path.join(HTML_FOLDER, FILE), "r") as f:
    file = json.load(f)

In [14]:
def compare_HTML_with_Markdown():
    with open(os.path.join(HTML_FOLDER, FILE), "r") as f:
        file = json.load(f)
        print(file["hasPart"][0]["text"])
    with open(os.path.join(SOURCE_FOLDER, FILE), "r") as f:
        file = json.load(f)
        print(file["hasPart"][0]["text"])

compare_HTML_with_Markdown()

<h1>Politique de cookies</h1>


<!-- Legal document generated by Complianz | GDPR/CCPA Cookie Consent https://wordpress.org/plugins/complianz-gdpr -->
<div id="cmplz-document" class="cmplz-document cookie-statement cmplz-document-eu"><p><i>Cette politique de cookies a été mise à jour pour la dernière fois le 21/11/2023 et s’applique aux citoyens et aux résidents permanents légaux de l’Espace Économique Européen et de la Suisse.</i><br></p><h2>1. Introduction</h2><p>Notre site web, <a href="https://www.ladrome.fr">https://www.ladrome.fr</a> (ci-après : « le site web ») utilise des cookies et autres technologies liées (par simplification, toutes ces technologies sont désignées par le terme « cookies »). Des cookies sont également placés par des tierces parties que nous avons engagées. Dans le document ci-dessous, nous vous informons de l’utilisation des cookies sur notre site web.</p><h2>2. Que sont les cookies ?</h2><p>Un cookie est un petit fichier simple envoyé avec les pages de ce si

FileNotFoundError: [Errno 2] No such file or directory: '../data/INSALyon/insalyon-2023-12-13-page/ladrome-page-581681.json'

We see that the parsing of the HTML file removed the h2 titles (they are converted as "\n\n**here is the title text**") which makes the splitting difficult. Keepeing a style like "##here is h2 title" would be better

## New chunking method based on titles

In [60]:
import os
from src.HTMLChunkNorris import HTMLChunkNorris

In [61]:
html_cn = HTMLChunkNorris()

## LaDrome
SOURCE_FOLDER = "../../data/D26/ladrome-2023-12-22-page/"

# FILE = "ladrome-page-13830.json"
# FILE = "ladrome-page-10516.json"
# FILE = "ladrome-page-61504.json"
# FILE = "ladrome-page-504967.json"
# FILE = "ladrome-page-567239.json"
# FILE = "ladrome-page-13718.json"
# FILE = "ladrome-page-10516.json"
# FILE = "ladrome-page-547344.json" # big chunks
# FILE = "ladrome-page-574603.json" # no title, just a list of links
# FILE = "ladrome-page-581681.json" # plein de petits titres
# FILE = "ladrome-page-10787.json"
FILE = "ladrome-page-11020.json"

# INSALyon
# SOURCE_FOLDER = "../../data/INSALyon/insalyon-2023-12-13-page/"

# FILE = "rejoindreinsalyon-page-500.json"
# FILE = "rejoindreinsalyon-page-160.json"
# FILE = "rejoindreinsalyon-page-4314.json"
# FILE = "rejoindreinsalyon-page-3908.json"

file = HTMLChunkNorris.read_json_file(os.path.join(SOURCE_FOLDER, FILE))
# print(file)
titles = html_cn.get_toc(file)
print(titles)
#chunks = html_cn.get_chunks(titles, FILE)
# print(chunks)
chunks = html_cn(
    os.path.join(SOURCE_FOLDER, FILE),
    max_title_level_to_use="h3",
    max_chunk_word_length=200,
    link_placement="in_sentence",
    chunk_tokens_exceeded_handling="split"
    )
print("\n======================================\n")
for c in chunks:
    print(c["text"])

[{'id': -1, 'text': '', 'level': -1, 'start_position': -1, 'end_position': -1, 'children': [{'id': 0, 'text': 'Les stations de la Drôme', 'level': 0, 'start_position': 0, 'end_position': 49}], 'parents': [], 'content': ''}, {'id': 0, 'text': 'Les stations de la Drôme', 'level': 0, 'start_position': 0, 'end_position': 49, 'children': [], 'parents': [{'id': -1, 'text': '', 'level': -1, 'start_position': -1, 'end_position': -1}], 'content': 'Ce site permet à l’usager de savoir si les stations de la drôme sont ouvertes ou non et d’avoir d’autres informations concernant les stations de ski de la drôme. Les stations de la Drôme'}]


# Les stations de la Drôme
Ce site permet à l’usager de savoir si les stations de la drôme sont ouvertes ou non et d’avoir d’autres informations concernant les stations de ski de la drôme. Les stations de la Drôme



In [7]:
import pandas as pd 

SOURCE_FOLDER = "../../data/D26/ladrome-2023-12-22-page/"
html_cn = HTMLChunkNorris()
file_list = os.listdir(SOURCE_FOLDER)
df_constructor = []
for file in file_list:
    chunks = html_cn(
        os.path.join(SOURCE_FOLDER, file),
        max_title_level_to_use="h3",
        max_chunk_word_length=250,
        link_placement="in_sentence",
        chunk_tokens_exceeded_handling="split"
    )
    df_constructor.extend(chunks)

drome_df = pd.DataFrame(df_constructor)
drome_df.head()

,id,token_count,word_count,source_file,text
0,ladrome-page-10511-0.json,657,361,ladrome-page-10511.json,# Un peu d’histoire\nSituée à mi-chemin entre ...
1,ladrome-page-10513-0.json,5,3,ladrome-page-10513.json,# Les cantons\n
2,ladrome-page-10516-0.json,91,43,ladrome-page-10516.json,# Filières d’excellence\n### Agriculture bio\n...
3,ladrome-page-10516-1.json,147,62,ladrome-page-10516.json,# Filières d’excellence\n### Gastronomie\nL’ex...
4,ladrome-page-10516-2.json,108,47,ladrome-page-10516.json,# Filières d’excellence\n### Œnologie\nLien en...


In [8]:
drome_df["word_count"].describe()

count     377.000000
mean      137.564987
std       164.841709
min         2.000000
25%        57.000000
50%        93.000000
75%       163.000000
max      1897.000000
Name: word_count, dtype: float64

In [9]:
sorted_chunks = drome_df.sort_values(by="word_count")
sorted_chunks.tail(10)

,id,token_count,word_count,source_file,text
182,ladrome-page-13832-0.json,1371,539,ladrome-page-13832.json,# Consultations réglementaires des CLI\nLes CL...
216,ladrome-page-19076-0.json,969,570,ladrome-page-19076.json,# Missions\nLes prérogatives des CLI ont été d...
308,ladrome-page-566648-0.json,1213,577,ladrome-page-566648.json,"# Alimentation\n#### Mieux manger, c’est mange..."
82,ladrome-page-10791-0.json,1456,634,ladrome-page-10791.json,# Les aides de la Maison Départementale de l’A...
279,ladrome-page-547344-1.json,1563,713,ladrome-page-547344.json,# FAQ – futur collège Mercurol-Veaunes\n### Se...
358,ladrome-page-582176-1.json,1350,742,ladrome-page-582176.json,# Expressions politiques\n### Octobre-Décembre...
281,ladrome-page-547344-3.json,1899,969,ladrome-page-547344.json,# FAQ – futur collège Mercurol-Veaunes\n### En...
145,ladrome-page-11061-0.json,3750,1021,ladrome-page-11061.json,# Archives des actes administratifs\nA compter...
280,ladrome-page-547344-2.json,2103,1136,ladrome-page-547344.json,# FAQ – futur collège Mercurol-Veaunes\n### Tr...
320,ladrome-page-574603-2.json,6383,1897,ladrome-page-574603.json,# Presse\n### Année 2023\nDécembre Fin des tra...


In [10]:
sorted_chunks.head(50)

,id,token_count,word_count,source_file,text
74,ladrome-page-10773-0.json,4,2,ladrome-page-10773.json,# Handicap\n
1,ladrome-page-10513-0.json,5,3,ladrome-page-10513.json,# Les cantons\n
130,ladrome-page-10993-0.json,7,3,ladrome-page-10993.json,# Le nucléaire\n
89,ladrome-page-10866-0.json,6,4,ladrome-page-10866.json,# Vivre à domicile\n
98,ladrome-page-10905-0.json,8,4,ladrome-page-10905.json,# Vivre en établissement\n
112,ladrome-page-10944-0.json,8,4,ladrome-page-10944.json,# Le laboratoire départemental\n
231,ladrome-page-43852-0.json,7,4,ladrome-page-43852.json,# Menu des collèges\n
75,ladrome-page-10775-0.json,8,5,ladrome-page-10775.json,# Le handicap chez l’enfant\n
139,ladrome-page-11047-0.json,16,6,ladrome-page-11047.json,# Actualités\n[archives type=monthly limit=20 ...
144,ladrome-page-11059-0.json,13,6,ladrome-page-11059.json,# Délibérations\n#### Articles non trouvés\n


In [11]:
import json

MIN_CHUNK_SIZE = 15 # chunks with less words than that will we discarded

DESTINATION_FOLDER = "../../data/D26/ladrome-2023-12-22-page-chunked-v1/"
if not os.path.exists(DESTINATION_FOLDER):
    os.mkdir(DESTINATION_FOLDER)

def save_record_as_json(record):
    with open(os.path.join(DESTINATION_FOLDER, record["id"]), "w") as f:
        json.dump(record, f, ensure_ascii=False)

drome_df = drome_df[drome_df["word_count"] >= MIN_CHUNK_SIZE]
for record in drome_df.to_dict(orient="records"):
    save_record_as_json(record)

In [12]:
import random

DESTINATION_FOLDER = "../../data/D26/ladrome-2023-12-22-page-chunked-v1/"
FILES = os.listdir(DESTINATION_FOLDER)
RANDOM_FILE = random.choice(FILES)
print(f"---{RANDOM_FILE}---")
with open(os.path.join(DESTINATION_FOLDER, RANDOM_FILE), "r") as f:
    tmp = json.load(f)
    for k,v in tmp.items():
        if k != "text":
            print(f"{k} : {v}")
        else:
            print("Texte:")
            print(v)

---ladrome-page-10970-0.json---
id : ladrome-page-10970-0.json
token_count : 102
word_count : 57
source_file : ladrome-page-10970.json
Texte:
# Les collections départementales
Plus de 900 œuvres et objets d’art, photographies et arts graphiques constituent les collections départementales. Elles soulignent la diversité des créations des artisans d’art et des artistes, notamment des photographes qui portent un regard personnel sur les patrimoines bâtis ou naturels de la Drôme. Découvrez ici quelques oeuvres photographiques appartenant aux collections départementales



In [5]:
import os 
import json
import pandas as pd

CHUNKS_FOLDER = "../data/D26/ladrome-2023-12-22-page-chunked-v1/"
df_constructor = []
for file in os.listdir(CHUNKS_FOLDER):
    with open(os.path.join(CHUNKS_FOLDER, file)) as f:
        df_constructor.append(json.load(f))

drome_df = pd.DataFrame(df_constructor)
drome_df.head()

,id,token_count,word_count,source_file,text
0,ladrome-page-10511-0.json,657,361,ladrome-page-10511.json,# Un peu d’histoire\nSituée à mi-chemin entre ...
1,ladrome-page-10516-0.json,91,43,ladrome-page-10516.json,# Filières d’excellence\n### Agriculture bio\n...
2,ladrome-page-10516-1.json,147,62,ladrome-page-10516.json,# Filières d’excellence\n### Gastronomie\nL’ex...
3,ladrome-page-10516-2.json,108,47,ladrome-page-10516.json,# Filières d’excellence\n### Œnologie\nLien en...
4,ladrome-page-10516-3.json,130,58,ladrome-page-10516.json,# Filières d’excellence\n### Cinéma d’animatio...
